In [1]:
%matplotlib inline
import importlib
from pytorch_utils_oh_2 import *

Pytorch utils oh: pytorch_utils_oh_2.py
Pytorch: 0.2.0_4


In [2]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [3]:
MODEL_SAVE_PATH = 'whole_gen_2_chars'

In [87]:
import pytorch_utils_oh_2; importlib.reload(pytorch_utils_oh_2); from pytorch_utils_oh_2 import *;

Pytorch utils oh: pytorch_utils_oh_2.py
Pytorch: 0.2.0_4


# Data loading

In [4]:
all_data = pickle.load(open("data/en_train_fixed_1.pkl", "rb" ))
all_data_sentence_index = all_data.set_index('sentence_id')

In [5]:
categories_all = sorted(all_data["class"].unique())
print(categories_all)
print(len(categories_all))
categories_index = dict((c, i) for i, c in enumerate(categories_all))

['ELECTRONIC', 'LETTERS', 'NOT_CHANGED', 'NUMBERS', 'PLAIN', 'VERBATIM']
6


### Utils stuff

In [6]:
chars_normal, chars_normal_index = load_characters_pkl('data/en_features/chars_normal.pkl')
print(''.join(chars_normal))

<SOS><EOS>☒ !"#$%&'(),-./0123456789:;ABCDEFGHIJKLMNOPQRSTUVWXYZ_abcdefghijklmnopqrstuvwxyz~£¥ª²³µº¼½¾éɒʻˈΩμ—€⅓⅔⅛


In [7]:
common_words, common_words_index = load_common_words_10k()
len(common_words)
common_words[0:10]

8192

['<EOS>', '<SOS>', '<UNK>', '<0000>', '<SAMPLE>', '.', ',', 'the', '"', 'of']

### More balanced sample

In [14]:
sample_data = all_data[all_data['class'] != 'NOT_CHANGED']
print("Data rows: {},  (dropped rows: {})".format(len(sample_data), len(all_data)-len(sample_data)))
sample_data = sample_data.reset_index(drop=True)

Data rows: 659544,  (dropped rows: 9258648)


In [10]:
sample_data.sample(5)

,sentence_id,token_id,class,before,after,class_org
127619,145610,1,LETTERS,UK,u k,LETTERS
584752,663616,6,LETTERS,DAA,d a a,LETTERS
504034,573024,19,NUMBERS,2006,two thousand six,DATE
261994,299035,7,NUMBERS,"4,300,000 m3",four million three hundred thousand cubic meters,MEASURE
269986,308494,22,NUMBERS,3,three,CARDINAL


balanced_data = sample_data

balanced_data_length = len(balanced_data)
def balanced_data_sample_row():
    global balanced_data_last_sample
    balanced_data_last_sample = balanced_data.iloc[random.randint(1, balanced_data_length-1)]
    return balanced_data_last_sample

In [84]:
balanced_data_classes_select = list(sample_data.groupby('class'))

balanced_data_accessed_counter = 0 
balanced_data_length = 0
def balanced_data_randomize(max_len=20000):
    global balanced_data, balanced_data_length, balanced_data_accessed_counter
    balanced_data = pd.concat([v.sample(min(max_len, len(v))) for k, v in balanced_data_classes_select])
    balanced_data_length = len(balanced_data)
    balanced_data_accessed_counter = 0

def balanced_data_sample_row():
    global balanced_data_accessed_counter
    global balanced_data_last_sample
    balanced_data_accessed_counter += 1
    if balanced_data_accessed_counter/balanced_data_length > 0.2:
        balanced_data_randomize()
    balanced_data_last_sample = balanced_data.iloc[random.randint(1, balanced_data_length-1)]
    return balanced_data_last_sample
    
balanced_data_randomize()

In [72]:
%%timeit
balanced_data_sample_row()

101 µs ± 1.5 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [83]:
#all_data.groupby('class')['class'].count()
#sample_data.groupby('class')['class'].count()
balanced_data.groupby('class')['class'].count()

class
ELECTRONIC     4964
LETTERS       20000
NUMBERS       20000
PLAIN         20000
VERBATIM      16950
Name: class, dtype: int64

In [12]:
 balanced_data_sample_row()

sentence_id     452978
token_id             2
class          LETTERS
before            C.A.
after              c a
class_org      LETTERS
Name: 395310, dtype: object

### Electronic words

In [15]:
tmp = list(set(sample_data['after']))
chars_after = sorted(list(set(''.join(tmp))))
chars_after = [SOS_TOKEN, EOS_TOKEN, UNKNOWN_CHAR] + chars_after
chars_after_index = dict((c, i) for i, c in enumerate(chars_after))
len(chars_after)
'_'.join(chars_after)
chars_after_index['z']

32

"<SOS>_<EOS>_☒_ _'_a_b_c_d_e_f_g_h_i_j_k_l_m_n_o_p_q_r_s_t_u_v_w_x_y_z_é"

30

In [93]:
pickle.dump(chars_after, open('data/en_features/chars_after_1.pkl', 'wb'))

In [94]:
chars_after = pickle.load(open('data/en_features/chars_after_1.pkl', 'rb'))
chars_after_index = dict((c, i) for i, c in enumerate(chars_after))

In [16]:
def after_string_to_tensor(word, include_eos=True):
    return string_to_tensor(word, chars_index=chars_after_index, include_eos=include_eos)
after_string_to_tensor('abcé').shape
after_string_to_tensor('abcé')[0, -2, -1] == 1

torch.Size([1, 5, 32])

True

In [17]:
onehot_sos = after_string_to_tensor([SOS_TOKEN], include_eos=False)
onehot_sos.size()

torch.Size([1, 1, 32])

### Samples

In [24]:
def get_random_sample():
    sample_row = balanced_data_sample_row()
    sentence_id = sample_row['class']

    rows = all_data_sentence_index.loc[sample_row['sentence_id']]
    befores = list(rows.before)
        
    token_id_idx = list(rows['token_id']).index(sample_row['token_id'])
    befores[token_id_idx] = SAMPLE_WORD_TOKEN
    
    return sample_row['before'], sample_row['after'], sample_row['class'], befores
            
def tmp():
    s_bef, s_aft, s_class, s_sentence = get_random_sample()
    print(s_class, ':', s_bef, '->', s_aft)
    print(' '.join(s_sentence))
    print(s_sentence)
    print(words_to_tensor(list(s_sentence), common_words_index).shape)
    print(string_to_tensor(s_bef, chars_normal_index).shape)
    print(after_string_to_tensor(s_aft.split(' ')).shape)
tmp()

NUMBERS : 1996 -> nineteen ninety six
He used this time to study law at the University of Indonesia , graduating in <SAMPLE> .
['He', 'used', 'this', 'time', 'to', 'study', 'law', 'at', 'the', 'University', 'of', 'Indonesia', ',', 'graduating', 'in', '<SAMPLE>', '.']
torch.Size([1, 18, 8192])
torch.Size([1, 5, 104])
torch.Size([1, 4, 32])


# Model functions

In [25]:
use_cuda = True

### Encoder

In [26]:
class EncoderRNN(nn.Module):
    def __init__(self, words_input_size, chars_input_size, words_hidden_size, chars_hidden_size,
                 words_layers=1, chars_layers=1):
        super(EncoderRNN, self).__init__()
        
        self.words_layers = words_layers
        self.chars_layers = chars_layers
        self.words_hidden_size = words_hidden_size
        self.chars_hidden_size = chars_hidden_size

        self.rnn_words = nn.LSTM(words_input_size, words_hidden_size // 2, words_layers,
                                 batch_first=True, bidirectional=True)

        self.rnn_chars = nn.LSTM(chars_input_size, chars_hidden_size // 2, chars_layers,
                                batch_first=True, bidirectional=True)
        
    def forward(self, word_vectors, string_tensor, hidden = None, init_hidden = True):
        if init_hidden:
            hidden_words, hidden_chars = self.init_hidden()
        
        all_outputs_words, hidden_words = self.rnn_words(word_vectors, hidden_words)
        output_words = all_outputs_words[:, -1]
        
        all_outputs_chars, hidden_chars = self.rnn_chars(string_tensor, hidden_chars)
        output_chars = all_outputs_chars[:, -1]
        
        output = torch.cat((output_words, output_chars), 1)
        
        return output

    def init_hidden(self):
        var1_1 = Variable(torch.zeros(2 * self.words_layers, 1, self.words_hidden_size // 2))
        var1_2 = Variable(torch.zeros(2 * self.words_layers, 1, self.words_hidden_size // 2))
        var2_1 = Variable(torch.zeros(2 * self.chars_layers, 1, self.chars_hidden_size // 2))
        var2_2 = Variable(torch.zeros(2 * self.chars_layers, 1, self.chars_hidden_size // 2))
        
        var1_1 = var1_1.cuda(); var1_2 = var1_2.cuda()
        var2_1 = var2_1.cuda(); var2_2 = var2_2.cuda()
        return ((var1_1, var1_2), (var2_1, var2_2))

In [27]:
encoder_rnn = EncoderRNN(words_input_size=len(common_words), chars_input_size=len(chars_normal),
                         words_hidden_size=128, chars_hidden_size=128,
                         words_layers=2, chars_layers=2).cuda()
encoder_rnn

EncoderRNN (
  (rnn_words): LSTM(8192, 64, num_layers=2, batch_first=True, bidirectional=True)
  (rnn_chars): LSTM(104, 64, num_layers=2, batch_first=True, bidirectional=True)
)

In [28]:
def test_encoder_single_sample():
    s_bef, s_aft, s_class, s_sentence = get_random_sample()
    
    words_t = words_to_tensor(list(s_sentence), common_words_index)
    words_t = Variable(words_t).cuda()
    
    string_t = string_to_tensor(s_bef, chars_normal_index)
    string_t = Variable(string_t).cuda()
    
    return encoder_rnn(words_t, string_t)
    
encoder_output = test_encoder_single_sample()
encoder_output.size()

torch.Size([1, 256])

### Decoder

In [30]:
class DecoderRNN(nn.Module):
    def __init__(self, input_size, hidden_size, n_layers=1):
        super(DecoderRNN, self).__init__()
        self.n_layers = n_layers
        self.hidden_size = hidden_size
        
        self.rnn = nn.GRU(input_size, hidden_size, n_layers,
                                 batch_first=True, bidirectional=False)
        
        self.lin_out = nn.Linear(hidden_size, input_size)
        #self.softmax = nn.LogSoftmax()

    def forward(self, char, hidden):
        #char = char.view(1,1,-1)
        #hidden = hidden.view(1,1,-1)
        output, hidden = self.rnn(char, hidden)
        output = output[:, -1] # view(1,-1)
        output = self.lin_out(output)
        output = F.log_softmax(output)
        return output, hidden
    
    def init_rest_hidden(self, input_var):
        hid_var = Variable(torch.zeros(self.n_layers - 1, 1, self.hidden_size)).cuda()
        res = torch.cat((input_var, hid_var), 0)
        return res
        

decoder_rnn = DecoderRNN(input_size=len(chars_after), hidden_size=encoder_output.size()[-1], n_layers=2)
decoder_rnn = decoder_rnn.cuda()
decoder_rnn

DecoderRNN (
  (rnn): GRU(32, 256, num_layers=2, batch_first=True)
  (lin_out): Linear (256 -> 32)
)

In [35]:
tmp_hiddens = decoder_rnn.init_rest_hidden(encoder_output.view(1,1,-1))
tmp_a, tmp_b = decoder_rnn(Variable(onehot_sos).cuda(), tmp_hiddens)
print(tmp_a.size())
print(tmp_b.size())
print(tmp_a.topk(1)[1])
print(chars_after[tmp_a.topk(1)[1].data[0][0]])

torch.Size([1, 32])
torch.Size([2, 1, 256])
Variable containing:
 6
[torch.cuda.LongTensor of size 1x1 (GPU 0)]

b


# Training etc

### Accuracy

In [88]:
def test_model_single_sample(model=None):
    s_bef, s_aft, s_class, s_sentence = sample = get_random_sample()
        
    #words_t = words_to_tensor(list(s_sentence), common_words_index)
    words_t = words_to_tensor(sentence_arr_tokenize(s_sentence), common_words_index)
    words_t = Variable(words_t).cuda()
    
    string_t = string_to_tensor(s_bef, chars_normal_index)
    string_t = Variable(string_t).cuda()
    
    encoder_output = encoder_rnn(words_t, string_t)
    
    encoder_output = encoder_output.view(1,1,-1)
    
    decoder_hidden = decoder_rnn.init_rest_hidden(encoder_output)
    decoder_input = Variable(onehot_sos).cuda()

    decoded_output = []
    max_length = 20
    for _ in range(max_length):
        decoder_output, decoder_hidden = decoder_rnn(decoder_input, decoder_hidden)
        #return decoder_output

        topv, topi = decoder_output.data.topk(1)
        char_index = topi[0][0]
        char = chars_after[char_index] # Use own prediction as next input
                
        if char == EOS_TOKEN:
            break

        decoded_output.append(char)
        
        decoder_input = after_string_to_tensor([char], include_eos=False)
        decoder_input = Variable(decoder_input).cuda()
    
    output = ''.join(decoded_output)
    return output, output, s_aft, sample
    
tmp = test_model_single_sample(None)
tmp

('p e a a c s',
 'p e a a c s',
 "p i a a's",
 ("PIAA's",
  "p i a a's",
  'LETTERS',
  ['More',
   'locally',
   ',',
   'Great',
   'Valley',
   'is',
   'a',
   'member',
   'of',
   'the',
   'Ches',
   'Mont',
   'League',
   'and',
   '<SAMPLE>',
   'District',
   'One',
   'region',
   '.']))

In [40]:
def print_local_wrong_predictions(max_results=10):
    arr = get_some_wrong_predictions(None, test_model_single_sample, max_iterations=10000, max_results=max_results)
    for sample, predict, output in arr:
        s_bef, s_aft, s_class, s_sentence = sample
        print("{:<14} => {:<14} || {} \n{:>17} {}".format(s_bef, predict, s_aft, '', ' '.join(s_sentence), ))


In [41]:
print_local_wrong_predictions(2)

TV-            => b b b b b b b b b b b b b b b b b b b b || t v 
                  Named Best Defenseman at <SAMPLE> pucken in 1990 .
2014           => b b b b b b b b b b b b b b b b b b b b || twenty fourteen 
                  In 2013 , Slattery directed his first feature film , God's Pocket ( <SAMPLE> ) , which he co wrote with Alex Metcalf .


In [42]:
%%time
test_model_accuracy(encoder_rnn, test_model_single_sample)

Accuracy: 0.00% (       0/   10000)
CPU times: user 10min, sys: 4.84 s, total: 10min 5s
Wall time: 2min 31s


0.0

### Training

In [89]:
def train(s_bef, s_aft, s_sentence, encoder_optimizer, decoder_optimizer, loss_function,
          use_teacher_forcing, max_length=20):
    
    #words_t = words_to_tensor(list(s_sentence), common_words_index)
    words_t = words_to_tensor(sentence_arr_tokenize(s_sentence), common_words_index)
    words_t = Variable(words_t).cuda()
    
    string_t = string_to_tensor(s_bef, chars_normal_index)
    string_t = Variable(string_t).cuda()
    
    encoder_output = encoder_rnn(words_t, string_t)
    encoder_output = encoder_output.view(1,1,-1)
    
    decoder_hidden = decoder_rnn.init_rest_hidden(encoder_output)
    decoder_input = Variable(onehot_sos).cuda()
    
    ###
    
    target_arr = list(s_aft) + [EOS_TOKEN]
    
    encoder_optimizer.zero_grad()
    decoder_optimizer.zero_grad()
    loss = 0
    
    decoded_output = []
    for i in range(len(target_arr)):
        decoder_output, decoder_hidden = decoder_rnn(decoder_input, decoder_hidden)

        decoder_target_i = chars_after_index[target_arr[i]]
        decoder_target_i = Variable(torch.LongTensor([decoder_target_i])).cuda()
        loss += loss_function(decoder_output, decoder_target_i)
        
        topv, topi = decoder_output.data.topk(1)
        char_index = topi[0][0]
        char = chars_after[char_index] # Use own prediction as next input
        decoded_output.append(char)
        
        if use_teacher_forcing:
            char = target_arr[i] # replace input with right target
        else:
            # use output normally as input 
            if char == EOS_TOKEN:
                break
                
        decoder_input = after_string_to_tensor([char], include_eos=False)
        decoder_input = Variable(decoder_input).cuda()
        
    if decoded_output[-1] == EOS_TOKEN:
        decoded_output = decoded_output[:-1]
        
    loss.backward()

    encoder_optimizer.step()
    decoder_optimizer.step()

    return ''.join(decoded_output), (loss.data[0] / len(target_arr))


In [64]:
def train_iterations(n_iters=100000, lr=0.001, teacher_forcing_ratio=0.5,
                     print_every=10000, plot_every=1000):

    start = time.time()
    
    decoder_rnn.train()
    encoder_rnn.train()

    current_loss = 0
    current_loss_iter = 0

    encoder_optimizer = torch.optim.Adam(encoder_rnn.parameters(), lr=lr)
    decoder_optimizer = torch.optim.Adam(decoder_rnn.parameters(), lr=lr)
    loss_function = nn.NLLLoss()
    
    for iteration in range(1, n_iters + 1):
        model_training.iterations += 1
        
        use_teacher_forcing = random.random() < teacher_forcing_ratio
        
        s_bef, s_aft, s_class, s_sentence = get_random_sample()
        
        result, loss = train(s_bef=s_bef, s_aft=s_aft, s_sentence=s_sentence,
                             encoder_optimizer=encoder_optimizer, decoder_optimizer=decoder_optimizer,
                             loss_function=nn.NLLLoss(), use_teacher_forcing=use_teacher_forcing,
                             max_length=40 )
        
        current_loss += loss
        current_loss_iter += 1

        # Print iter number, loss, name and guess
        if iteration % print_every == 0:
            teacher_forcing_str = ""
            if use_teacher_forcing:
                teacher_forcing_str = "(forcing)"
            correct = '✓' if result == s_aft else "✗: {}".format(s_aft)
            
            print("{:>6d} {:>4.0%} ({:>8}) {:>7.3f}   | {:>6.2f}: {} -> {} ({}) {}".format(
                      model_training.iterations, iteration/n_iters, time_since(start),
                      current_loss/current_loss_iter, loss,
                      s_bef, result, correct, teacher_forcing_str))

        # Add current loss avg to list of losses
        if iteration % plot_every == 0:
            model_training.losses.append(current_loss / plot_every)
            model_training.learning_rates.append(lr)
            current_loss = 0
            current_loss_iter = 0
            
        if model_training.iterations % 50000 == 0 or model_training.iterations == 10:
            model_training.save_models()
            acc = test_model_accuracy(encoder_rnn, test_model_single_sample)
            model_training.accuracy.append(acc)
    
    # test_model_accuracy(model, n_sample=10000)

In [65]:
model_training = ModelTraining(MODEL_SAVE_PATH, [encoder_rnn, decoder_rnn])

Save path: data/models/whole_gen_2_chars


In [66]:
train_iterations(n_iters=50, print_every=9, lr=0.0001)

     9  18% (   0m 0s)   3.449   |   3.42: 109 -> bb                (✗: one hundred nine) (forcing)
Saved model to data/models/whole_gen_2_chars/10_(EncoderRNN/DecoderRNN)
Accuracy: 0.00% (       0/   10000)
    18  36% (  2m 26s)   3.433   |   3.39: 2009 -> n                  (✗: two thousand nine) 
    27  54% (  2m 27s)   3.418   |   3.40: ABL -> nn     (✗: a b l) 
    36  72% (  2m 27s)   3.406   |   3.43: VC -> n    (✗: v c) 
    45  90% (  2m 27s)   3.390   |   3.27: LTCN -> n        (✗: l t c n) 


In [67]:
train_iterations(n_iters=(1000-model_training.iterations), print_every=500, lr=0.0001)

   550  53% (  0m 19s)   2.923   |   3.53: pp -> teee (✗: p p) (forcing)


In [68]:
train_iterations(n_iters=9000, lr=0.0001, print_every=1000)

  2000  11% (  0m 39s)   2.668   |   2.83: $14,758 -> tin teen t n t e t                                   (✗: fourteen thousand seven hundred fifty eight dollars) 
  3000  22% (  1m 16s)   2.339   |   2.25: 107 -> tie t n     t ne   (✗: one hundred seven) (forcing)
  4000  33% (  1m 52s)   2.099   |   1.58: 2004 -> two thousand tieet (✗: two thousand four) 
  5000  44% (  2m 28s)   1.976   |   2.08: 30 September 1946 -> the teenty twenty twenty tieety tie (✗: the thirtieth of september nineteen forty six) 
  6000  56% (   3m 4s)   1.898   |   1.59: December 2000 -> tineteen twe thousand  (✗: december two thousand) (forcing)
  7000  67% (  3m 39s)   1.839   |   2.74: J. -> s  (✗: j) 
  8000  78% (  4m 16s)   1.824   |   2.30: 149 -> twe thousand eine (✗: one hundred forty nine) 
  9000  89% (  4m 52s)   1.751   |   1.61: 1880 -> nineteen sixty t (✗: eighteen eighty) 
 10000 100% (  5m 28s)   1.709   |   2.41: Theatre -> torrree  (✗: theater) (forcing)


In [69]:
train_iterations(n_iters=90000, print_every=10000)

 20000  11% (  5m 59s)   1.246   |   1.65: KL -> c k (✗: k l) (forcing)
 30000  22% (  12m 5s)   0.919   |   0.07: 19 -> nineteen (✓) (forcing)
 40000  33% ( 18m 44s)   0.727   |   0.14: ABC -> a b c (✓) (forcing)
 50000  44% ( 25m 30s)   0.540   |   0.78: IAC -> i c c (✗: i a c) (forcing)
Saved model to data/models/whole_gen_2_chars/50000_(EncoderRNN/DecoderRNN)
Accuracy: 53.66% (    5366/   10000)
 60000  56% (  34m 6s)   0.499   |   0.04: 1926 -> nineteen twenty six (✓) (forcing)
 70000  67% ( 40m 34s)   0.426   |   0.02: 1957 -> nineteen fifty seven (✓) 
 80000  78% (  47m 1s)   0.384   |   0.02: 1977 -> nineteen seventy seven (✓) 
 90000  89% ( 53m 37s)   0.341   |   0.01: 14 -> fourteen (✓) 
100000 100% ( 60m 17s)   0.328   |   0.19: bros -> brothers (✓) (forcing)
Saved model to data/models/whole_gen_2_chars/100000_(EncoderRNN/DecoderRNN)
Accuracy: 62.66% (    6266/   10000)


In [70]:
train_iterations(n_iters=300000, print_every=10000, teacher_forcing_ratio=0.5, lr=0.001)

110000   3% (  6m 33s)   0.321   |   0.05: 2009 -> two thousand nine (✓) 
120000   7% (  13m 5s)   0.303   |   0.07: III -> three (✓) 
130000  10% ( 19m 46s)   0.320   |   0.14: 3rd -> third (✓) (forcing)
140000  13% ( 26m 27s)   0.304   |   0.06: 1,  -> one (✓) (forcing)


KeyboardInterrupt: 

In [ ]:
# Using sampled infos from now on

In [85]:
train_iterations(n_iters=300000, print_every=10000, teacher_forcing_ratio=0.5, lr=0.001)

Saved model to data/models/whole_gen_2_chars/150000_(EncoderRNN/DecoderRNN)
Accuracy: 66.54% (    6654/   10000)
159924   3% (   7m 7s)   0.357   |   0.00: - -> to (✓) 
169924   7% (  13m 8s)   0.338   |   0.01: 1950 -> nineteen fifty (✓) (forcing)
179924  10% ( 18m 29s)   0.428   |   0.17: specialising -> specializing (✓) 
189924  13% ( 23m 44s)   0.410   |   0.01: : -> to (✓) (forcing)
199924  17% ( 28m 57s)   0.460   |   0.53: February 23, 2012 -> saneuary twenty third owenyy eeelve (✗: february twenty third twenty twelve) (forcing)
Saved model to data/models/whole_gen_2_chars/200000_(EncoderRNN/DecoderRNN)
Accuracy: 70.00% (    7000/   10000)
209924  20% ( 35m 39s)   0.418   |   2.27: χ -> omic (✗: chi) 
219924  23% ( 40m 49s)   0.407   |   0.00: - -> to (✓) (forcing)
229924  27% ( 46m 14s)   0.470   |   0.01: ff -> f f (✓) (forcing)
239924  30% ( 51m 25s)   0.500   |   0.01: J. -> j (✓) 
249924  33% ( 56m 39s)   0.465   |   0.63: labourer -> laboreb (✗: laborer) (forcing)
Saved mo

KeyboardInterrupt: 

In [92]:
train_iterations(n_iters=300000, print_every=10000, teacher_forcing_ratio=0.5, lr=0.001)

IndexError: list index out of range

In [ ]:
print_local_wrong_predictions()

In [ ]:
train_iterations(n_iters=100000, print_every=10000, teacher_forcing_ratio=0.5, lr=0.0001)

In [ ]:
print_local_wrong_predictions()

In [ ]:
train_iterations(n_iters=200000, print_every=10000, teacher_forcing_ratio=0.3, lr=0.001)

In [ ]:
print_local_wrong_predictions()

In [ ]:
train_iterations(n_iters=100000, print_every=10000, teacher_forcing_ratio=0, lr=0.001)

In [ ]:
print_local_wrong_predictions()

In [ ]:
train_iterations(n_iters=300000, print_every=10000, teacher_forcing_ratio=0, lr=0.0001)

In [ ]:
print_local_wrong_predictions()

In [ ]:
train_iterations(n_iters=200000, print_every=10000, teacher_forcing_ratio=0, lr=0.0001)

In [ ]:
print_local_wrong_predictions()

In [ ]:
train_iterations(n_iters=200000, print_every=10000, teacher_forcing_ratio=0, lr=0.0001)

In [ ]:
print_local_wrong_predictions()

In [ ]:
time.sleep(7*60*60)

In [ ]:
train_iterations(n_iters=500000, print_every=10000, teacher_forcing_ratio=0, lr=0.0001)

In [ ]:
print_local_wrong_predictions()

In [ ]:
train_iterations(n_iters=500000, print_every=10000, teacher_forcing_ratio=0, lr=0.0001)

In [ ]:
print_local_wrong_predictions()